In [ ]:
import json
import base64
import pandas as pd
import numpy as np
import subprocess
import os
import boto3
import tarfile
from tensorflow.keras.models import load_model
from sklearn.preprocessing import RobustScaler

s3 = boto3.client('s3')

def download_model_from_s3(s3_uri, local_path):
    bucket, key = s3_uri.lstrip('s3://').split('/', 1)
    s3.download_file(bucket, key, local_path)

def predict_ecg_class(event, context):
    # Extract the image file from the multipart/form-data
    image_file = event['files']['img'][0]

    # Save the image to a temporary file
    temp_image_path = '/tmp/uploaded_image.png'
    with open(temp_image_path, 'wb') as f:
        f.write(image_file['content'])

    # Run plotdigitizer and preprocess the custom DataFrame
    command = f'plotdigitizer "{temp_image_path}" -p 0,0 -p 2,0 -p 0,1 -l 2,29 -l 4,5 -l 22,5'
    output = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    temp_csv_path = os.path.splitext(temp_image_path)[0] + ".traj.csv"
    df = pd.read_csv(temp_csv_path, sep=' ', header=None)
    robust_scaler = RobustScaler()
    df = pd.DataFrame(robust_scaler.fit_transform(df), columns=df.columns)
    df = df.iloc[:, 1:2]
    df = df.head(10).T.reset_index(drop=True)
    df.columns = ['0_pre-RR', '0_pPeak', '0_rPeak', '0_sPeak', '0_qt_interval', '1_pre-RR', '1_qPeak', '1_qt_interval', '1_qrs_morph0', '1_qrs_morph1']
    
    # Assuming your model expects input with shape (batch_size, num_features)
    custom_data = df.values.reshape((1, -1))

    # Load the pre-trained model from S3
    s3_model_uri = 's3://psutgrad/ecg_classifier.tar.gz'
    model_local_path = '/tmp/ecg_classifier.tar.gz'
    download_model_from_s3(s3_model_uri, model_local_path)
    
    # Extract the model from the tar.gz file
    with tarfile.open(model_local_path, 'r:gz') as tar:
        tar.extractall('/tmp/')

    # Load the model
    model_path = '/tmp/ecg_classifier.h5'
    model = load_model(model_path)

    # Make predictions
    predictions = model.predict(custom_data)

    # Display predictions or use them as needed
    predicted_class_index = np.argmax(predictions)
    class_mapping = {
        0: 'Normal Beat',
        1: 'Ventricular Beat',
        2: 'Supraventricular Beat',
        3: 'Fusion Beat'
    }

    predicted_class_label = class_mapping.get(predicted_class_index, 'Unknown')

    return {
        'statusCode': 200,
        'body': json.dumps({'predicted_class_label': predicted_class_label})
    }
